In [ ]:
!pip install numpyro

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.8/360.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
import os
import math
import numpy as np

import time

import jax
import jax.numpy as jnp
from jax import random, lax, jit, ops
from jax.example_libraries import stax

import numpyro
from numpyro.infer import SVI, MCMC, NUTS, init_to_median, Predictive, RenyiELBO
import numpyro.distributions as dist

import geopandas as gpd
import plotly.express as px

from termcolor import colored

import pickle

jax.config.update("jax_default_device", jax.devices()[1])
print(f"Jax using device : {jax.devices()}")

ModuleNotFoundError: No module named 'aggGP'

In [ ]:
#define the functions necessary
def dist_euclid(x, z):
    """
    Computes Eucledian Distance Between Regions. This function is used by
    exp_sq_kernel function (kernel function for gaussian processes)
    """
    x = jnp.array(x) # (ngrid_pts, lat/lon) <- i.e (7304,2)
    z = jnp.array(z) # (ngrid_pts, lat/lon) <- i.e (7304,2)
    if len(x.shape)==1:
        x = x.reshape(x.shape[0], 1) #(2618,) -> (7304,1)
    if len(z.shape)==1:
        z = x.reshape(x.shape[0], 1) #(2618,) -> (7304,1)
    n_x, m = x.shape # 7304 , 2
    n_z, m_z = z.shape # 7304 , 2
    assert m == m_z
    delta = jnp.zeros((n_x,n_z)) #(ngrid_pts,ngrid_pts) <- i.e (7304,7304)
    for d in jnp.arange(m):
        x_d = x[:,d] #(ngrid_pts-lat/lon,) <- (7304,)
        z_d = z[:,d] #(ngrid_pts-lat/lon,) <- (7304,)
        delta += (x_d[:,jnp.newaxis] - z_d)**2 # (7304,7304)

    return jnp.sqrt(delta) #(7304,7304)

def exp_sq_kernel(x, z, var, length, noise, jitter=1.0e-4):
    dist = dist_euclid(x, z) #(7304, 7304)
    deltaXsq = jnp.power(dist/ length, 2.0)
    k = var * jnp.exp(-0.5 * deltaXsq)
    k += (noise + jitter) * jnp.eye(x.shape[0])
    return k # (ngrid_pts, ngrid_pts) <- (7304,7304)

def M_g(M, g):
    '''
    - $M$ is a matrix with binary entries $m_{ij},$ showing whether point $j$ is in polygon $i$
    - $g$ is a vector of GP draws over grid
    - $maltmul(M, g)$ gives a vector of sums over each polygon
    '''
    M = jnp.array(M)
    g = jnp.array(g).T
    return(jnp.matmul(M, g))

#AggVAE Model

##Function for Predictive Simulation (Prior)

In [ ]:
def gp_aggr(args):
    x = args["x"] # (num_grid_pts, lat+lon)
    gp_kernel = args["gp_kernel"]
    noise = args["noise"]

    M= args["M"]

    #kernal hyperparams
    kernal_length = args["kernel_length"]
    kernel_var = args["kernel_var"]

    # Random effect - aggregated GP
    length = numpyro.sample("kernel_length", kernal_length) #(,)
    var = numpyro.sample("kernel_var",kernel_var) #(,)
    # Kernel for allgrid points
    k = gp_kernel(x,x,var, length, noise) #(num_grig_pts,num_grid_pts)
    # GP draw evaluated at all ... grid pints
    f = numpyro.sample(
        "f",
        dist.MultivariateNormal(loc = jnp.zeros(x.shape[0]), covariance_matrix = k)
        ) #(num_grid_pts,)

    #aggregate f into gp_aggr according to indexing of (point in polygon)
    gp_aggr = numpyro.deterministic("gp_aggr", M_g(M, f)) #(num_regions,)

    return gp_aggr

##Define the VAE

In [ ]:
def vae_encoder(hidden_dim = 50, z_dim = 40):
    return stax.serial(
        #(num_samples, num_regions) -> (num_samples, hidden_dims)
        stax.Dense(hidden_dim, W_init = stax.randn()), #i.e(5,58) -> (5,50)
        stax.Elu,
        stax.FanOut(2),
        stax.parallel(
            # mean : (num_samples, hidden_dim) -> (num_samples, z_dim)
            stax.Dense(z_dim, W_init = stax.randn()), #(5,50) -> (5,40)
            #std : (num_samples, hidden_dim) -> (num_samples, z_dim)
            stax.serial(stax.Dense(z_dim, W_init = stax.randn()), stax.Exp) #(5,50) -> (5,40)
        )
    )

def vae_decoder(hidden_dim, out_dim):
    return stax.serial(
        # (num_samples, z_dim) -> (num_samples, hidden_dim): (5,40) -> (5,50)
        stax.Dense(hidden_dim, W_init = stax.randn()),
        stax.Elu,
        # (num_samples, hidden_dim) -> (num_samples, num_regions) : (5,50) -> (5, 58)
        stax.Dense(out_dim, W_init = stax.randn())
    )


def vae_model(batch, hidden_dim, z_dim):
    """This computes the decoder portion"""
    batch = jnp.reshape(batch, (batch.shape[0], -1)) # (num_samples, num_regions) <- i.e (5,58)
    batch_dim, out_dim = jnp.shape(batch) # 5 , 58

    # vae-decoder in numpyro module
    decode = numpyro.module(
        name = "decoder",
        nn = vae_decoder(hidden_dim = hidden_dim, out_dim = out_dim),
        input_shape = (batch_dim, z_dim) #(5,40)
    )

    # Sample a univariate normal
    #! ISSUE HERE : lax.sub cannot broadcast shapes (5,40) & (40,) here
    #! SO HAD TO CHANGE dist.Normal(jnp.zeros((z_dim,)), jnp.ones((z_dim,))) TO THIS dist.Normal(jnp.zeros((5,z_dim)), jnp.ones((5,z_dim)))
    z = numpyro.sample(
        "z",
        dist.Normal(
            jnp.zeros((batch_dim,z_dim)),
            jnp.ones((batch_dim,z_dim))
            )
    ) # (z_dim,) : i.e (40,)
    # Forward pass from decoder
    gen_loc = decode(z) #(num_regions,) : (58,)
    obs = numpyro.sample(
        "obs",
        dist.Normal(gen_loc, args["vae_var"]),
        obs = batch
    ) #(num_samples, num_regions) : (5,58)
    return obs


def vae_guide(batch, hidden_dim, z_dim):
    """This computes the encoder portion"""
    batch = jnp.reshape(batch, (batch.shape[0], -1)) #(num_samples, num_regions) : (5,58)
    batch_dim, input_dim = jnp.shape(batch)# num_samples , num_regions : 5 , 58

    # vae-encoder in numpyro module
    encode = numpyro.module(
        name = "encoder",
        nn = vae_encoder(hidden_dim=hidden_dim,z_dim = z_dim),
        input_shape = (batch_dim, input_dim) #(5,58)
    ) #(num_samples, num_regions) -> (num_samples, hidden_dims) : i.e (5,58) -> (5,40)

    # Samapling mu, sigma - Pretty much the forward pass
    z_loc, z_std = encode(batch) #mu : (num_samples, z_dim), sigma2 : (num_samples, z_dim) <- (5,40),(5,40)
    # Sample a value z based on mu and sigma
    z = numpyro.sample("z", dist.Normal(z_loc, z_std)) #(num_sample, z_dim) : (5,40)
    return z

##Train the VAE encoder

In [ ]:
@jax.jit
def epoch_train(rng_key, svi_state, num_train):
    def body_fn(i, val):
        rng_key_i = jax.random.fold_in(rng_key, i) #Array(2,)
        rng_key_i, rng_key_ls, rng_key_var, rng_key_noise = jax.random.split(rng_key_i, 4) #Tuple(Array(2,) x 4)
        loss_sum, svi_state = val #val --svi_state

        batch = agg_gp_predictive(rng_key_i, args)["gp_aggr"] #(5,116) <- num_samples : 5, total_districts : 116
        #* svi is where the vae_model & vae_guide gets applied
        svi_state, loss = svi.update(svi_state, batch)
        loss_sum += loss / args["batch_size"]
        return loss_sum, svi_state

    return lax.fori_loop(lower = 0, upper = num_train, body_fun=body_fn, init_val=(0.0, svi_state))

@jax.jit
def eval_test(rng_key, svi_state, num_test):
    def body_fn(i, loss_sum):
        rng_key_i = jax.random.fold_in(rng_key, i)
        rng_key_i, rng_key_ls, rng_key_varm, rng_key_noise = jax.random.split(rng_key_i, 4)
        batch = agg_gp_predictive(rng_key_i, args)["gp_aggr"]
        #* svi is where the vae_model & vae_guide gets applied
        loss = svi.evaluate(svi_state, batch) / args["batch_size"]
        loss_sum += loss
        return loss_sum

    loss = lax.fori_loop(lower = 0, upper = num_test,body_fun =  body_fn, init_val = 0.0)
    loss = loss / num_test
    return loss


##Function to plot the GP

In [ ]:
def plot_process(gp_draws):
    p = px.line()
    for i in range(len(gp_draws)):
        p.add_scatter(x = np.arange(gp_draws.shape[1]), y = gp_draws[i, :])

    p.update_traces(line_color = "black")
    p.update_layout(
        template = "plotly_white",
        xaxis_title = "region", yaxis_title = "num cases",
        showlegend = False)
    p.show()


##Load the variables

In [ ]:
# Lat/Lon Values of artificial grid
x = np.load("lat_lon_x.npy")
pol_pts_all = np.load("pol_pts_all.npy")
pt_which_pol_all = np.load("pt_which_pol_all.npy")

df_combined = gpd.read_file("final_district_divisions.shp")

##Arguments

In [ ]:
args = {
        "x": x,
        "gp_kernel": exp_sq_kernel,
        "noise": 1e-4,
        "M": pol_pts_all,

        # VAE training
        "rng_key": random.PRNGKey(5),
        "num_epochs": 20,
        #"learning_rate": 1.0e-3,
        "learning_rate": 0.0005,
        "batch_size": 100,
        "hidden_dim": 50,
        "z_dim": 40,
        "num_train": 100,
        "num_test":100,
        "vae_var": 1,
        # kernel hyperparams
        "kernel_length" : dist.InverseGamma(3,3), #!hyperparam
        "kernel_var" : dist.HalfNormal(0.05) #!hyperparam
    }


##Prior predictive simulation

In [ ]:
rng_key, rng_key_ = random.split(random.PRNGKey(4))
agg_gp_predictive = Predictive(gp_aggr,num_samples = 5)

agg_gp_draws = agg_gp_predictive(rng_key_, args)["gp_aggr"] #(num_samples, num_regions) <- (5,58)

In [ ]:
# Plotting
plot_process(agg_gp_draws)

##Initiate Training Loop

In [ ]:
adam = numpyro.optim.Adam(step_size = args["learning_rate"])
svi = SVI(
        vae_model,
        vae_guide,
        adam,
        RenyiELBO(),
        hidden_dim = args["hidden_dim"],
        z_dim = args["z_dim"]
    )

rng_key, rng_key_samp, rng_key_init = random.split(args["rng_key"],3)
#(num_samples, num_regions)
init_batch = agg_gp_predictive(rng_key_, args)["gp_aggr"] #(num_samples, num_regions) <- i.e (5,58)
svi_state = svi.init(rng_key_init, init_batch)

test_loss_list = []
for i in range(args["num_epochs"]):
    rng_key, rng_key_train, rng_key_test, rng_key_infer = random.split(rng_key, 4)
    t_start = time.time()

    num_train = 1000
    # Where forward/backward pass gets called for train
    train_loss , svi_state = epoch_train(rng_key_train, svi_state, num_train)

    num_test = 1000
    # Where forward/backward pass gets called for test
    test_loss = eval_test(rng_key_test, svi_state, num_test)
    test_loss_list += [test_loss]

    print("Epoch : {}, train loss : {:.2f}, test loss : {:.2f} ({:.2f} s.)".format(i, train_loss, test_loss, time.time() - t_start))

    if math.isnan(test_loss):
        break


In [ ]:
#check initial SVI state
print("svi_state:", svi_state.optim_state)

##Save the decoder

In [ ]:
# save decoder
decoder_params = svi.get_params(svi_state)
save_path = f"model_weights/aggVAE_e{args['num_epochs']}_h{args['hidden_dim']}_z{args['z_dim']}"

with open(save_path, "wb") as file:
    pickle.dump(decoder_params, file)